In [1]:
import os
import cv2
import torch
import torchvision
from PIL import Image
import time
import numpy as np
from pydantic import BaseModel
import pandas as pd

In [2]:
model_dice = torch.hub.load("ultralytics/yolov5", "custom", path="best.pt", force_reload=False, device="cuda" if torch.cuda.is_available() else "cpu")
model_dice.conf = 0.6
model_dice.iou = 0.01

Using cache found in C:\Users\fudawei/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-2-17 torch 1.10.2+cpu CPU

Fusing layers... 
Model Summary: 232 layers, 7260003 parameters, 0 gradients
Adding AutoShape... 


In [3]:
model_test = torch.hub.load("ultralytics/yolov5", "yolov5s")

Using cache found in C:\Users\fudawei/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-2-17 torch 1.10.2+cpu CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [25]:
cap = cv2.VideoCapture(0)

In [24]:
if not cap.isOpened():
    raise IOError("Cannot open")

OSError: Cannot open

In [26]:
while True:
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #print(frame.shape)
    #detect = model(frame)
    #detect.render()
    #for img in detect.imgs:
        #img = frame
        #cv2.imshow("output", frame)
        #image = Image.fromarray(img)
        #image.show(image)
    #time.sleep(1)
    cv2.imshow("input", frame)
    c = cv2.waitKey(1)
    if c == 27:
        break


KeyboardInterrupt: 

In [27]:
#ret, frame = cap.read()
#cv2.imshow("Input", frame)
cap.release()
cv2.destroyAllWindows()

In [3]:
img = cv2.imread(r"WIN_20220215_15_37_24_Pro.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [4]:
img = Image.open("WIN_20220215_15_37_24_Pro.jpg")

In [5]:
result = model(img)

C:\Users\fudawei\Anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [6]:
#result.render()
result.show()

In [28]:
type(img)

numpy.ndarray

In [28]:
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
image.show(image)

In [112]:
result.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,656.965942,2454.889160,843.334595,2634.050049,0.914676,3,4
1,785.053467,1153.781494,948.521790,1314.674316,0.904330,0,1
2,1139.915771,2341.445312,1325.351562,2513.143311,0.898576,0,1
3,2297.536865,2186.107910,2457.154785,2327.589600,0.896783,4,5
4,1502.370605,2617.429199,1676.412842,2788.376709,0.896279,1,2
5,1642.406128,789.658569,1798.434937,937.577087,0.886416,3,4
6,707.199585,1883.278931,880.764099,2049.059570,0.886046,0,1
7,1170.532959,2018.976074,1336.167847,2180.095703,0.883724,1,2
8,556.366638,2187.046387,718.900269,2334.593750,0.872420,0,1
9,1486.517700,859.192444,1648.670654,1009.107361,0.872234,0,1


In [30]:
img.shape

(480, 640, 3)

In [ ]:
cv2.destroyAllWindows()

In [6]:
cap.release()

In [4]:
import io
import base64
import json
import uvicorn
import nest_asyncio
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

In [5]:
class StreamInput(BaseModel):
    b64str: str
    model: str

In [6]:
app = FastAPI(title="Dice detection")

In [7]:
@app.get("/")
def home():
    return "Server is running. Go to http://localhost:8080/docs"

In [8]:
@app.post("/file/to-img")
def detect_return_image(image: UploadFile):
    
    image_stream = io.BytesIO(image.file.read())
    image_stream.seek(0)
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    result = model_test(image)
    result.render()
    for img in result.imgs:
        detected = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    _, encoded = cv2.imencode(".jpg", detected)
    byte_output = encoded.tobytes()
    
    return StreamingResponse(io.BytesIO(byte_output), media_type="image/jpg")
    
@app.post("/file/to-b64")
def detect_return_base64(image: UploadFile):
    
    image_stream = io.BytesIO(image.file.read())
    image_stream.seek(0)
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    result = model_test(image)
    result.render()
    for img in result.imgs:
        buffered = io.BytesIO()
        img_object = Image.fromarray(img)
        img_object.save(buffered, format="JPEG")
        img_base64 = base64.b64encode(buffered.getvalue())
    
    return img_base64


@app.post("/file/to-json")
def detect_return_labels(image: UploadFile):
    
    image_stream = io.BytesIO(image.file.read())
    image_stream.seek(0)
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = model_test(image)
    labels = result.pandas().xyxy[0].to_json(orient="records")
    
    labels_json = json.loads(labels)
    
    return labels_json

In [9]:
@app.post("/predict/to-img")
def detect_return_image(client_input: StreamInput):
    image_bytes = base64.b64decode(client_input.b64str.encode("utf-8"))
    image_array = np.frombuffer(image_bytes, dtype=np.uint8) 
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if client_input.model == "Dice":
        result = model_dice(image)
    else:
        result = model_test(image)
    result.render()
    for img in result.imgs:
        detected = img#cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    _, encoded = cv2.imencode(".jpg", detected)
    byte_output = encoded.tobytes()
    
    return StreamingResponse(io.BytesIO(byte_output), media_type="image/jpg")
    
@app.post("/predict/to-b64")
def detect_return_base64(client_input: StreamInput):
    image_bytes = base64.b64decode(client_input.b64str.encode("utf-8"))
    image_array = np.frombuffer(image_bytes, dtype=np.uint8) 
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if client_input.model == "Dice":
        result = model_dice(image)
    else:
        result = model_test(image)
    result.render()
    for img in result.imgs:
        buffered = io.BytesIO()
        img_object = Image.fromarray(img)
        img_object.save(buffered, format="JPEG")
        img_base64 = base64.b64encode(buffered.getvalue())
    
    return img_base64


@app.post("/predict/to-json")
def detect_return_labels(client_input: StreamInput):
    image_bytes = base64.b64decode(client_input.b64str.encode("utf-8"))
    image_array = np.frombuffer(image_bytes, dtype=np.uint8) 
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if client_input.model == "Dice":
        result = model_dice(image)
    else:
        result = model_test(image)
    
    df = result.pandas().xyxy[0]
    labels = df.to_json(orient="records")
    labels_json = json.loads(labels)

    return labels_json

In [14]:
nest_asyncio.apply()

host = "0.0.0.0" #if os.getenv("DOCKER-SETUP") else "127.2.0.1"

uvicorn.run(app, host=host, port=80)

INFO:     Started server process [21660]
Started server process [21660]
INFO:     Waiting for application startup.
Waiting for application startup.
INFO:     Application startup complete.
Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:80 (Press CTRL+C to quit)
Uvicorn running on http://0.0.0.0:80 (Press CTRL+C to quit)
INFO:     Shutting down
Shutting down
INFO:     Waiting for application shutdown.
Waiting for application shutdown.
INFO:     Application shutdown complete.
Application shutdown complete.
INFO:     Finished server process [21660]
Finished server process [21660]


In [10]:
os.getenv("DOCKER-SETUP")

In [33]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.
